# PyTorch MNIST

References:
- [https://nextjournal.com/gkoehler/pytorch-mnist](https://nextjournal.com/gkoehler/pytorch-mnist)
- [https://www.youtube.com/watch?v=Vs730jsRgO8](https://www.youtube.com/watch?v=Vs730jsRgO8)

TODO:
- Refactor
- Understand model inputs/ outputs
- data augmentation
- ONNX
- ONNX Js
- Retrain with new data?

https://onnx.ai/
https://github.com/microsoft/onnxruntime/tree/main/js/web#readme  
https://pytorch.org/tutorials/advanced/super_resolution_with_onnxruntime.html  
https://pytorch.org/tutorials/beginner/introyt.html  
https://pytorch.org/tutorials/beginner/pytorch_with_examples.html  

- balanced dataset
- normalized
- augmentation
- over fitting
- under fitting
- shuffle

## Dependencies

In [7]:
%pip install torch torchvision

Note: you may need to restart the kernel to use updated packages.


In [8]:
import torch
import torchvision

import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

import matplotlib.pyplot as plt

In [9]:

train_batch_size = 10
test_batch_size = 10
data_directory = 'data'

transformer = torchvision.transforms.Compose([torchvision.transforms.ToTensor()])

train_resources = torchvision.datasets.MNIST(data_directory, train=True, download=True, transform=transformer)
test_resources = torchvision.datasets.MNIST(data_directory, train=False, download=True, transform=transformer)

train_dataset = torch.utils.data.DataLoader(train_resources, batch_size=train_batch_size, shuffle=True)
test_dataset= torch.utils.data.DataLoader(test_resources, batch_size=test_batch_size, shuffle=True)

In [10]:
len(train_resources)

60000

In [11]:
class Net(nn.Module):
	def __init__(self):
		super(Net, self).__init__()
		self.fc1 = nn.Linear(28 * 28, 64)
		self.fc2 = nn.Linear(64, 64)
		self.fc3 = nn.Linear(64, 64)
		self.fc4 = nn.Linear(64, 10)

	def forward(self, x):
		x = F.relu(self.fc1(x))
		x = F.relu(self.fc2(x))
		x = F.relu(self.fc3(x))
		x = self.fc4(x)
		return F.softmax(x, dim=1)

net = Net()

In [12]:
optimizer = optim.Adam(net.parameters(), lr=0.001)
loss_function = nn.CrossEntropyLoss()

epochs = 5

for epoch in range(epochs):
	for X, y in train_dataset:
		net.zero_grad()
		X = X.view(-1, 28 * 28)
		output = net(X)

		loss = loss_function(output, y)
		loss.backward()
		optimizer.step()

	print(loss)

tensor(1.4725, grad_fn=<NllLossBackward0>)
tensor(1.4653, grad_fn=<NllLossBackward0>)
tensor(1.4612, grad_fn=<NllLossBackward0>)
tensor(1.4612, grad_fn=<NllLossBackward0>)
tensor(1.5611, grad_fn=<NllLossBackward0>)


In [13]:
correct = 0
total = 0

with torch.no_grad():
	for X, y in test_dataset:
		X = X.view(-1, 28 * 28)
		output = net(X)

		for index, predictions in enumerate(output):
			if torch.argmax(predictions) == y[index]:
				correct += 1
			
			total += 1

print('Accuracy:', round(correct/total, 3))
torch.save(net.state_dict(), 'output/model.pt')

Accuracy: 0.942


In [14]:
net = Net()
net.load_state_dict(torch.load('output/model.pt'))
net.eval()

dummy_input = torch.zeros(1, 28 * 28)
torch.onnx.export(net, dummy_input, 'output/onnx_model.onnx', verbose=True)

RuntimeError: mat1 and mat2 shapes cannot be multiplied (1x78400 and 784x64)